<a href="https://colab.research.google.com/github/Jamaleldin/MRNET-for-Knee-Diagnosis/blob/master/MRNet_models_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **All needed library imports**

In [0]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.models import Model
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


# **All needed labels and constants**

In [0]:
# Here we define the labels to be used in obtaining the data
# series
axial = 'axial';
coronal = 'coronal';
sagittal = 'sagittal';
# data set
valid = 'valid';
train = 'train';
# symptom
acl = 'acl';
meniscal = 'meniscus';
abnormal = 'abnormal';
# models
vgg = 'VGG16';
inception = 'Inception V3';
resnet = 'Resnet';
# model types
extractor = 'Extractor';
classifier = 'Classifier';
regressor = 'Regressor';
# paths
path_data = '/content/drive/My Drive/MRNET data set/MRNet-v1.0';
path_model = '/content/drive/My Drive/Models';
delim = '/';
# extensions
extension_model = '.h5';
extension_numpy = '.npy';
extension_csv = '.csv';
# total data
total_train = 1130;
total_test = 120;

# **Obtaining the data for training in a correct format**

In [0]:
# importing the drive to obtain the data files
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
# unzipping the data zip file run once
!unzip '/content/drive/My Drive/MRNET data set/MRNet-v1.0.zip' -d '/content/drive/My Drive/MRNET data set'

In [3]:
# This function is used to obtain the data from drive to memory for training
def get_data_train(series,anomaly,start = 0,batch_size = total_train):
  input_data = [];
  for exam_no in range(start,batch_size + start):
      path_input = path_data + delim  + train + delim + series + delim + format(exam_no,'04d') + extension_numpy;
      input_data.append(normalize(np.stack([np.load(path_input)] * 3, axis = 3)))
  path_output = path_data + delim + train + '-' + anomaly + extension_csv;
  output_data = np.genfromtxt(path_output, delimiter= ',')[:,1].astype(int);
  return input_data, output_data[start:batch_size + start];

NameError: ignored

In [2]:
# This function is used to obtain the data from drive to memory for testing
def get_data_test(series,anomaly,start = 0,batch_size = total_test):
  input_data = [];
  for iterator in range(start,batch_size + start):
      exam_no = iterator + total_train;
      path_input = path_data + delim  + valid + delim + series + delim + format(exam_no,'04d') + extension_numpy;
      input_data.append(normalize(np.stack([np.load(path_input)] * 3, axis = 3)))
  path_output = path_data + delim + valid + '-' + anomaly + extension_csv;
  output_data = np.genfromtxt(path_output, delimiter= ',')[:,1].astype(int);
  return input_data, output_data[start:batch_size + start];

NameError: ignored

# **Data normalization part**

In [0]:
# normalizing the image pixels to be a number from 0 to 1
def normalize(X):
  return X/255.;

# **Data augmentation**

In [0]:
# based on augmentation techniques defined in the paper
def get_generators():
  train_datagen = ImageDataGenerator(
      rotation_range=25,
      width_shift_range=[-25,25],
      height_shift_range=[-25,25],
      horizontal_flip=True);
  validation_datagen = ImageDataGenerator();
  return train_datagen,validation_datagen;

# **Data splitting part**

In [0]:
# splitting the data to train and validate with ratios 90% to 10%
def split(X , Y):
  return X[:1017],Y[:1017],X[1017:],Y[1017:];

# **Training functions:**

In [0]:
def train_extractor(model:Model,model_type,series,anomaly):
  # obtaining the data
  input_train,output_train = get_data_train(series,anomaly);
  input_train,output_train,input_validate,output_validate = split(input_train,output_train);
  # processing to train for the extractor specifically: obtaining only one image from each patient
  input_train = get_one_image(input_train);
  input_validate = get_one_image(input_validate);
  # performing data augmentation
  train_datagen,validation_datagen = get_generators();
  train_generator = train_datagen.flow(input_train,output_train, batch_size = 20);
  validation_generator = validation_datagen.flow(input_validate,output_validate, batch_size = 20);
  # defining the call backs for training
  save_path = path_model + delim + model_type + delim + extractor + delim+series + '_' + anomaly + extension_model;
  save_best = ModelCheckpoint(save_path, monitor='val_acc', mode='max', verbose=2, save_best_only=True);
  stop = EarlyStopping(monitor='val_loss', mode='min', verbose=2, patience=5);
  # performing a traininig operation with a batch size to overcome any overfitting
  training_history = model.fit_generator(train_generator, validation_data=validation_generator, epochs=50, callbacks=[save_best,stop]);
  # plotting the graph for the training history
  plot(training_history);
  return training_history;

In [0]:
def get_one_image(input_train):
  new_input_train = [];
  for curr in input_train:
     new_input_train.append(curr[0]);
  return np.array(new_input_train);

In [0]:
def train_classifier(extractor:Model,binary_classifier:Model,model_type,series,anomaly):
  # obtaining the data
  input_train,output_train = get_data_train(series,anomaly);
  input_train,output_train,input_validate,output_validate = split(input_train,output_train);
  # obtaining the data specifically for the classifier by getting the extracted features
  input_train = get_features(extractor,input_train);
  input_validate = get_features(extractor,input_validate);
  # defining the call backs for training
  save_path = path_model + delim + model_type + delim + classifier + delim + series + '_' + anomaly + extension_model;
  save_best = ModelCheckpoint(save_path, monitor='val_acc', mode='max', verbose=2, save_best_only=True);
  stop = EarlyStopping(monitor='val_loss', mode='min', verbose=2, patience=5);
  # performing a traininig operation with a batch size to overcome any overfitting
  training_history = binary_classifier.fit(x=input_train,y = output_train, validation_data=(input_validate,output_validate), epochs=50,batch_size = 20, callbacks=[save_best,stop]);
  # plotting the graph for the training history
  plot(training_history);
  return training_history;

In [0]:
def get_features(extractor:Model,input):
  # forming the extraction model
  model = keras.models.Sequential();
  model.add(extractor);
  model.add(keras.layers.GlobalAveragePooling2D(data_format='channels_last'));
  # extracting the features
  result = [];
  for curr in input:
    # getting the predictions
    features_extracted = model.predict(curr);
    # getting maximum between the batches
    max_features = np.max(features_extracted,axis=0);  
    result.append(max_features);
  return np.array(result);

In [0]:
def train_regressor(logistic_regressor:Model,extractor_axial:Model,extractor_sagittal:Model,extractor_coronal:Model,model_type,anomaly):
  # predictions of all classifiers:
  pred_axial,output_train,val_axial,output_validate = get_classifications(extractor_axial,model_type,axial,anomaly);
  pred_sagittal,output_train,val_sagittal,output_validate = get_classifications(extractor_sagittal,model_type,sagittal,anomaly);
  pred_coronal,output_train,val_coronal,output_validate = get_classifications(extractor_coronal,model_type,coronal,anomaly);
  # combining the predictions
  input_train = np.array([pred_axial,pred_coronal,pred_sagittal]).reshape(3,pred_axial.shape[0]).T;
  input_validate = np.array([val_axial,val_coronal,val_sagittal]).reshape(3,val_axial.shape[0]).T;
  # defining the call backs for training
  save_path = path_model + delim + model_type + delim + regressor + delim + anomaly + extension_model;
  save_best = ModelCheckpoint(save_path, monitor='val_acc', mode='max', verbose=2, save_best_only=True);
  stop = EarlyStopping(monitor='val_loss', mode='min', verbose=2, patience=5);
  # performing a traininig operation with a batch size to overcome any overfitting
  training_history = logistic_regressor.fit(x=input_train,y = output_train, validation_data=(input_validate,output_validate), epochs=50,batch_size = 20, callbacks=[save_best,stop]);
  # plotting the graph for the training history
  plot(training_history);
  return training_history;


In [0]:
def get_classifications(extractor:Model,model_type,series,anomaly):
  # obtaining the data
  input_train,output_train = get_data_train(series,anomaly);
  input_train,output_train,input_validate,output_validate = split(input_train,output_train);
  # obtaining the features
  input_train = get_features(extractor,input_train);
  input_validate = get_features(extractor,input_validate);
  # load the classifier
  binary_classifier = load_model(model_type,series,anomaly,classifier);
  # obtaining the cassifications
  input_train = binary_classifier.predict(input_train);
  input_validate = binary_classifier.predict(input_validate);
  return input_train,output_train,input_validate,output_validate;

# **Functions for model evaluation:**

In [0]:
def plot(history):
  # drawing the model graph
  pd.DataFrame(history.history).plot(figsize=(10, 7));
  plt.grid(True);
  plt.gca().set_ylim(0, 1);
  plt.show();

In [0]:
def test_extractor(model:Model,series,anomaly):
  # getting test data
  input_test,output_test = get_data_test(series,anomaly);
  # processing it to test the extractor specifically: getting one image from each patient only
  input_test = get_one_image(input_test);
  return model.evaluate(input_test,output_test);

In [0]:
def test_classifier(extractor:Model,classifier:Model,series,anomaly):
  # getting test data
  input_test,output_test = get_data_test(series,anomaly);
  # processing it to test the extractor specifically: getting one image from each patient only
  input_test = get_features(extractor,input_test);
  return classifier.evaluate(input_test,output_test);

In [0]:
def test_regressor(extractor_axial:Model,extractor_sagittal:Model,extractor_coronal:Model,model_type,anomaly):
  # getting the regressor
  logistic_regressor = load_model(model_type,anomaly,regressor);
  # getting the test classifications
  pred_axial,output_test = get_classifications_test(extractor_axial,model_type,axial,anomaly);
  pred_sagittal,output_test = get_classifications_test(extractor_sagittal,model_type,sagittal,anomaly);
  pred_coronal,output_test = get_classifications_test(extractor_coronal,model_type,coronal,anomaly);
  # combining the predictions
  input_test = np.array([pred_axial,pred_coronal,pred_sagittal]).reshape(3,pred_axial.shape[0]).T;
  return logistic_regressor.evaluate(input_test,output_test);


In [0]:
def get_classifications_test(extractor:Model,model_type,series,anomaly):
  # obtaining the data
  input_test,output_test = get_data_test(series,anomaly);
  # obtaining the features
  input_test = get_features(extractor,input_train);
  # load the classifier
  binary_classifier = load_model(model_type,series,anomaly,classifier);
  # obtaining the cassifications
  input_test = binary_classifier.predict(input_train);
  return input_test,output_test;

# **Functions to get the model predictions**

In [0]:
# extractors is a List of the saved extractors containing: axial-abnormal,sagittal-abnormal,coronal-abnormal,axial-acl,
# sagittal-acl coronal-acl,axial-meniscal,sagittal-meniscal,coronal-meniscal.
def predict(extractors:list,input_axial,input_sagittal,input_coronal,model_type):
  inputs = [input_axial,input_sagittal,input_coronal];
  # getting the prediction for each anomaly
  pred_abnormal = predict_anomaly(extractors[0:3],inputs,abnormal,model_type);
  pred_acl = predict_anomaly(extractors[3:6],inputs,acl,model_type);
  pred_meniscal = predict_anomaly(extractors[6:9],inputs,meniscal,model_type);
  # combining the predictions and multiplying by 100
  predictions = [pred_abnormal,pred_acl,pred_meniscal] * 100;
  # printing the predictions
  str_abnormal = abnormal + ' : ' +  str(predictions[0]) + '%';
  str_acl = acl + ' : ' + str(predictions[1]) + '%';
  str_meniscal = meniscal +' : ' + str(predictions[2]) + '%';
  print(str_abnormal);
  print(str_acl);
  print(str_meniscal);

In [0]:
def predict_anomaly(extractors:list,inputs:list,anomaly,model_type):
  # getting the classifiers
  classifier_axial = load_model(model_type,axial,anomaly,classifier);
  classifier_sagittal = load_model(model_type,sagittal,anomaly,classifier);
  classifier_coronal = load_model(model_type,coronal,anomaly,classifier);
  # getting the regressor
  logistic_regressor = load_regressor(model_type,anomaly,regressor);
  # getting classifications
  pred_axial = get_classification_single(extractors[0],classifier_axial,inputs[0]);
  pred_sagittal = get_classification_single(extractors[1],classifier_sagittal,inputs[1]);
  pred_coronal = get_classification_single(extractors[2],classifier_coronal,inputs[2]);
  # combining the predictions
  input_regressor = np.array([pred_axial,pred_coronal,pred_sagittal]).reshape(3,pred_axial.shape[0]).T;
  # final prediction is returned
  return logistic_regressor.predict(input_regressor);

In [0]:
def get_classification_single(extractor:Model,binary_classifier:Model,input_single):
  # obtaining the features
  result = get_features(extractor,input_single);
  # obtaining the cassifications
  result = binary_classifier.predict(result);
  return result;

# **Functions to manipualte saved models**

In [0]:
def load_model(model_type,series,anomaly,model_part):
  model_path = path_model + delim + model_type + delim + model_part + delim + series + '_' + anomaly + extension_model;
  return tf.keras.models.load_model(model_path);

In [0]:
def load_regressor(model_type,anomaly,model_part):
  model_path = path_model + delim + model_type + delim + model_part + delim + anomaly + extension_model;
  return tf.keras.models.load_model(model_path);